In [ ]:
!nvidia-smi
import torch, sys
print("Python:", sys.version)
print("Torch:", torch.__version__ if hasattr(torch, "__version__") else "Not installed")


Sun Nov  9 13:14:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Fresh start
%pip -q install --upgrade pip

# Install PyTorch CUDA 12.4 wheels
%pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
import torch, platform
print("Torch:", torch.__version__, "| CUDA available:", torch.cuda.is_available(), "| Device cap:", torch.cuda.get_device_capability() if torch.cuda.is_available() else None)


Torch: 2.5.1+cu124 | CUDA available: True | Device cap: (7, 5)


In [ ]:
%pip -q install unsloth


In [ ]:
# For Torch 2.5 + CUDA 12.4 wheels installed above:
%pip -q install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!wget -qO- https://raw.githubusercontent.com/unslothai/unsloth/main/unsloth/_auto_install.py | python -


pip install --upgrade pip && pip install "unsloth[cu124-torch250] @ git+https://github.com/unslothai/unsloth.git"


In [ ]:
%pip install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
%pip install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth-zoo.git


  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-req-build-pwjii9df
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-req-build-pwjii9df
  Resolved https://github.com/unslothai/unsloth.git to commit 1c0ad844f170f67c7cdf6f7a9465bafb0f9627df
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for unsloth: filename=unsloth-2025.11.2-py3-none-any.whl size=355531 sha256=b08544de79774b6aede66c3d28064b4ee0cbb0e366410e01d9c672f82ce729a0
  Stored in directory: /tmp/pip-ephem-wheel-cache-vulo7ctb/wheels/60/3e/1f/e576c07051d90cf64b6a41434d87ccf4db33fafd5343bf5de0
Successfully built unsloth
  Attempting uninstall: unsloth
    Found existing installation: unsloth 2025.11.2
    Uninstalling unsloth-2025.11.2:
      Successfully uninstalled unsloth-2025.11.2
  Cloning https://github.com/unslothai/unsloth-zoo.git to /tmp/pip-req-build-s

In [ ]:
from unsloth import FastLanguageModel, FastModel
import torch
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

max_seq_length = 2048

# Tiny public jsonl for a quick smoke test
url = "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"
dataset = load_dataset("json", data_files={"train": url}, split="train")

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gpt-oss-20b",   # pick any supported model
    max_seq_length = max_seq_length,
    load_in_4bit = True,                  # QLoRA 4-bit
    load_in_8bit = False,
    load_in_16bit = False,
    full_finetuning = False,
    # token="hf_..."                       # if using gated models
)

model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    max_seq_length = max_seq_length,
    use_rslora = False,
    loftq_config = None,
)

trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    tokenizer = tokenizer,
    args = SFTConfig(
        max_seq_length = max_seq_length,
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,           # use num_train_epochs=1..3 for a real run
        logging_steps = 1,
        output_dir = "outputs",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)

trainer.train()


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


ImportError: cannot import name 'is_mistral_common_available' from 'transformers.utils' (/usr/local/lib/python3.12/dist-packages/transformers/utils/__init__.py)

In [ ]:
!nvidia-smi
import sys; print("Python:", sys.version)


Sun Nov  9 13:20:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
%pip -q install --upgrade pip
%pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
import torch; print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU available:", torch.cuda.is_available())


Torch: 2.5.1+cu124 | CUDA: 12.4 | GPU available: True


In [ ]:
# 3a) Force-reinstall Unsloth core + zoo (recommended by Unsloth docs)
%pip -q install --upgrade --force-reinstall --no-cache-dir --no-deps unsloth unsloth_zoo

# 3b) Bring Transformers and friends to known-good versions
#    - Transformers: recent version with the utils symbols Unsloth expects
#    - TRL: version Unsloth’s patchers target
#    - PEFT/Accelerate/BitsAndBytes: common stack for SFT/QLoRA
%pip -q install "transformers==4.57.1" "trl==0.18.1" "peft>=0.13.2" "accelerate>=0.34.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0"

# (Optional) If you previously tried the Git install, undo conflicts by re-forcing:
# %pip -q install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git
# %pip -q install --upgrade --force-reinstall --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth-zoo.git


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.11.3 requires trl!=0.19.0,<=0.24.0,>=0.18.2, but you have trl 0.18.1 which is incompatible.
unsloth 2025.11.2 requires trl!=0.19.0,<=0.23.0,>=0.18.2, but you have trl 0.18.1 which is incompatible.


In [ ]:
# 0) Info (optional)
!nvidia-smi
import sys; print("Python:", sys.version)


Sun Nov  9 13:23:05 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# 1) Torch (CUDA 12.4 wheels are stable on Colab)
%pip -q install --upgrade pip
%pip -q install --index-url https://download.pytorch.org/whl/cu124 torch torchvision torchaudio
import torch; print("Torch:", torch.__version__, "| CUDA:", torch.version.cuda, "| GPU:", torch.cuda.is_available())


Torch: 2.5.1+cu124 | CUDA: 12.4 | GPU: True


In [ ]:
# Create a clean virtual environment
!python -m venv /content/unsloth-venv
# Upgrade pip inside the venv
!/content/unsloth-venv/bin/python -m pip install --upgrade pip


Error: Command '['/content/unsloth-venv/bin/python3', '-m', 'ensurepip', '--upgrade', '--default-pip']' returned non-zero exit status 1.
/content/unsloth-venv/bin/python: No module named pip


In [ ]:
!rm -rf /content/unsloth-venv


In [ ]:
!python -m venv /content/unsloth-venv --without-pip


In [ ]:
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/unsloth-venv/bin/python get-pip.py


  Using cached pip-25.3-py3-none-any.whl.metadata (4.7 kB)
Using cached pip-25.3-py3-none-any.whl (1.8 MB)


In [ ]:
# Torch (CUDA 12.4)
!/content/unsloth-venv/bin/python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124

# Unsloth + dependencies
!/content/unsloth-venv/bin/python -m pip install \
  "trl==0.23.0" "transformers==4.56.2" \
  "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0" \
  unsloth unsloth_zoo


Looking in indexes: https://download.pytorch.org/whl/cu124
  Using cached https://download.pytorch.org/whl/cu124/torch-2.5.1%2Bcu124-cp312-cp312-linux_x86_64.whl (908.2 MB)
  Using cached https://download.pytorch.org/whl/filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached https://download.pytorch.org/whl/typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
  Using cached https://download.pytorch.org/whl/networkx-3.5-py3-none-any.whl.metadata (6.3 kB)
  Using cached https://download.pytorch.org/whl/jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached https://download.pytorch.org/whl/fsspec-2025.9.0-py3-none-any.whl.metadata (10 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (24.6 MB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl (883 kB)
  Using cached https://download.pytorch.org/whl/cu124/nvidia_cuda_cup

In [ ]:
import os, sys, glob, site

# Path to your venv
venv_path = "/content/unsloth-venv"

# Compute the venv's site-packages folder
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"

# If folder name differs (e.g., python3.10 vs 3.12), auto-detect it
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]

# Add it so imports come from the venv
site.addsitedir(site_dir)
print("✅ Added venv to sys.path:", site_dir)

# ---- Sanity check ----
import torch, transformers, trl, unsloth
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("TRL:", trl.__version__)
print("Unsloth imported OK from:", unsloth.__file__)


✅ Added venv to sys.path: /content/unsloth-venv/lib/python3.12/site-packages


/tmp/ipython-input-400182861.py:20: UserWarning: WARNING: Unsloth should be imported before trl, transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import torch, transformers, trl, unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch: 2.8.0+cu128
Transformers: 4.57.1
TRL: 0.23.0
Unsloth imported OK from: /usr/local/lib/python3.12/dist-packages/unsloth/__init__.py


In [ ]:
!nvidia-smi
import torch
print("GPU available:", torch.cuda.is_available())


Sun Nov  9 13:45:21 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   53C    P0             28W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from unsloth import FastLanguageModel, FastModel
from trl import SFTTrainer, SFTConfig
from datasets import load_dataset

max_seq_length = 1024

# Small public dataset for a quick test
dataset = load_dataset(
    "json",
    data_files={"train": "https://huggingface.co/datasets/laion/OIG/resolve/main/unified_chip2.jsonl"},
    split="train[:200]"
)

# Load base model (you can later change to Llama, Gemma, etc.)
model, tokenizer = FastModel.from_pretrained(
    model_name      = "unsloth/gpt-oss-20b",
    max_seq_length  = max_seq_length,
    load_in_4bit    = True,      # fits on T4
    full_finetuning = False,
)

# Apply LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0,
    bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=max_seq_length,
)

# Configure trainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    args=SFTConfig(
        output_dir="outputs",
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        max_steps=40,      # quick run
        logging_steps=2,
        optim="adamw_8bit",
        seed=3407,
        max_seq_length=max_seq_length,
    ),
)

trainer.train()


unified_chip2.jsonl:   0%|          | 0.00/95.6M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

==((====))==  Unsloth 2025.11.2: Fast Gpt_Oss patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gpt_oss won't work! Using float32.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anuradhasrivastav25 (anuradhasrivastav25-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss
2,3.246800
4,2.982800
6,2.773100
8,2.457800
10,1.911100
12,1.672400
14,1.973900
16,2.150000
18,1.673500
20,1.773400


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=40, training_loss=2.0534964919090273, metrics={'train_runtime': 1153.5599, 'train_samples_per_second': 0.277, 'train_steps_per_second': 0.035, 'total_flos': 6124558737604608.0, 'train_loss': 2.0534964919090273, 'epoch': 1.6})

In [1]:
from datasets import load_dataset
ds = load_dataset("tatsu-lab/alpaca", split="train[:5000]")  # take small slice for speed
ds = ds.train_test_split(test_size=0.1, seed=42)
train_ds, eval_ds = ds["train"], ds["test"]

def format_alpaca(example):
    instruction = example["instruction"].strip()
    input_text = example["input"].strip()
    output = example["output"].strip()
    if input_text:
        prompt = f"### Instruction:\n{instruction}\n\n### Input:\n{input_text}\n\n### Response:\n{output}"
    else:
        prompt = f"### Instruction:\n{instruction}\n\n### Response:\n{output}"
    return {"text": prompt}

train_data = train_ds.map(format_alpaca, remove_columns=train_ds.column_names)
eval_data  = eval_ds.map(format_alpaca,  remove_columns=eval_ds.column_names)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [4]:
!/content/unsloth-venv/bin/python -m pip install unsloth unsloth_zoo


/bin/bash: line 1: /content/unsloth-venv/bin/python: No such file or directory


In [1]:
# === Recreate Unsloth venv cleanly ===
!python -m venv /content/unsloth-venv --without-pip
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/unsloth-venv/bin/python get-pip.py
!/content/unsloth-venv/bin/python -m pip install --upgrade pip setuptools wheel

# Install PyTorch (CUDA 12.4 wheel)
!/content/unsloth-venv/bin/python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124

# Install Unsloth + compatible stack
!/content/unsloth-venv/bin/python -m pip install \
  "trl==0.23.0" "transformers==4.56.2" \
  "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0" \
  unsloth unsloth_zoo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 14.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [setuptools]
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 14.1 MB/s  0:00:26
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 151.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.5 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 136.8 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 15.9 MB/s  0:00:19
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 18.5 MB/s  0:00:14
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 34.5 MB/s  0:00:06
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 68.0 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 66.6 MB/s  0:00:01
     ━━━━━━━━━━━

In [2]:
import os, sys, glob, site

venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]
site.addsitedir(site_dir)
print("✅ Added venv to sys.path:", site_dir)

# Sanity check
import torch, transformers, trl, unsloth
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("TRL:", trl.__version__)
print("Unsloth imported OK from:", unsloth.__file__)


✅ Added venv to sys.path: /content/unsloth-venv/lib/python3.12/site-packages


/tmp/ipython-input-2057073134.py:13: UserWarning: WARNING: Unsloth should be imported before trl, transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  import torch, transformers, trl, unsloth


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
Torch: 2.8.0+cu126
Transformers: 4.57.1
TRL: 0.23.0
Unsloth imported OK from: /content/unsloth-venv/lib/python3.12/site-packages/unsloth/__init__.py


In [3]:
from unsloth import FastLanguageModel, FastModel
from trl import SFTTrainer, SFTConfig
# ... rest of the fine-tuning code


Full Finetuning (smollm2-135m) on Alpaca subset

In [4]:
!nvidia-smi
import torch; print("CUDA:", torch.cuda.is_available())


Sun Nov  9 14:33:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             26W /   70W |     102MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
from datasets import load_dataset

# Small, fast slice for first run. Later increase to "train[:100%]".
raw = load_dataset("tatsu-lab/alpaca", split="train[:5000]", cache_dir="/content/.cache_hf")
splits = raw.train_test_split(test_size=0.1, seed=42)
train_ds, eval_ds = splits["train"], splits["test"]

def format_alpaca(ex):
    instr = (ex.get("instruction") or "").strip()
    inp   = (ex.get("input") or "").strip()
    out   = (ex.get("output") or "").strip()
    user  = f"### Instruction:\n{instr}" + (f"\n\n### Input:\n{inp}" if inp else "")
    return {"text": f"{user}\n\n### Response:\n{out}"}

train_data = train_ds.map(format_alpaca, remove_columns=train_ds.column_names)
eval_data  = eval_ds.map(format_alpaca,  remove_columns=eval_ds.column_names)

print(train_data[0]["text"][:500])
print(f"Train/Eval sizes: {len(train_data)}/{len(eval_data)}")


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/4500 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

### Instruction:
Classify this sentence as True or False.

### Input:
The capital of Spain is Madrid.

### Response:
True
Train/Eval sizes: 4500/500


In [6]:
from unsloth import FastModel
from trl import SFTTrainer, SFTConfig

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = False,     # FULL fine-tune (no quantization)
    full_finetuning = True,      # <— key difference vs LoRA
)

args = SFTConfig(
    output_dir="outputs_full",
    per_device_train_batch_size=8,     # small model → can go higher; drop if OOM
    gradient_accumulation_steps=1,
    num_train_epochs=1,                # or set max_steps=1000 for time-boxed training
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    optim="adamw_torch",               # standard AdamW
    learning_rate=2e-4,                # small model tolerates a bit higher LR
    weight_decay=0.01,
    seed=42,
    max_seq_length=MAX_LEN,
    dataset_text_field="text",
    fp16=True,                         # T4 = FP16 (if bf16=False)
    bf16=False,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=args,
)

trainer.train()


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Float16 full finetuning uses more memory since we upcast weights to float32.


model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/158 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: anuradhasrivastav25 (anuradhasrivastav25-san-jose-state-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Step,Training Loss,Validation Loss
200,1.720800,1.664466
400,1.635800,1.613969


Unsloth: Will smartly offload gradients to save VRAM!


TrainOutput(global_step=563, training_loss=1.6760839408074983, metrics={'train_runtime': 470.1511, 'train_samples_per_second': 9.571, 'train_steps_per_second': 1.197, 'total_flos': 503722991808000.0, 'train_loss': 1.6760839408074983, 'epoch': 1.0})

In [7]:
model.save_pretrained("finetuned_full_smollm2")
tokenizer.save_pretrained("finetuned_full_smollm2")
!ls -lah finetuned_full_smollm2


total 518M
drwxr-xr-x 2 root root 4.0K Nov  9 14:44 .
drwxr-xr-x 1 root root 4.0K Nov  9 14:43 ..
-rw-r--r-- 1 root root   29 Nov  9 14:44 added_tokens.json
-rw-r--r-- 1 root root  795 Nov  9 14:43 config.json
-rw-r--r-- 1 root root  168 Nov  9 14:43 generation_config.json
-rw-r--r-- 1 root root 456K Nov  9 14:44 merges.txt
-rw-r--r-- 1 root root 514M Nov  9 14:44 model.safetensors
-rw-r--r-- 1 root root  856 Nov  9 14:44 special_tokens_map.json
-rw-r--r-- 1 root root 4.0K Nov  9 14:44 tokenizer_config.json
-rw-r--r-- 1 root root 3.4M Nov  9 14:44 tokenizer.json
-rw-r--r-- 1 root root 782K Nov  9 14:44 vocab.json


In [8]:
prompt = "Explain CRISP-DM phases briefly for a grad-level data mining class."
inp = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    out = model.generate(**inp, max_new_tokens=160, do_sample=True, temperature=0.7, top_p=0.9)
print(tokenizer.decode(out[0], skip_special_tokens=True))


RuntimeError: index_copy_(): self and source expected to have the same dtype, but got (self) Float and (source) Half

Colab 2 (LoRA) on the same dataset (smaller memory, faster)

In [9]:
from unsloth import FastLanguageModel, FastModel
from trl import SFTTrainer, SFTConfig

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 1024

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,      # LoRA path (QLoRA)
    full_finetuning = False,     # <— enables PEFT
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)

args = SFTConfig(
    output_dir="outputs_lora",
    per_device_train_batch_size=8,        # should be fine; lower if OOM
    gradient_accumulation_steps=1,
    num_train_epochs=1,
    logging_steps=20,
    eval_strategy="steps",
    eval_steps=200,
    save_steps=200,
    save_total_limit=2,
    optim="adamw_8bit",                   # 8-bit optimizer
    learning_rate=2e-4,
    weight_decay=0.01,
    seed=42,
    max_seq_length=MAX_LEN,
    dataset_text_field="text",
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_data,
    eval_dataset=eval_data,
    args=args,
)
trainer.train()


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.


Unsloth: Making `model.base_model.model.model` require gradients


The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 4,500 | Num Epochs = 1 | Total steps = 563
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


Step,Training Loss,Validation Loss
200,1.730700,1.688717
400,1.680300,1.655471


TrainOutput(global_step=563, training_loss=1.768907067619039, metrics={'train_runtime': 411.9862, 'train_samples_per_second': 10.923, 'train_steps_per_second': 1.367, 'total_flos': 526890080448000.0, 'train_loss': 1.768907067619039, 'epoch': 1.0})

In [10]:
model.save_pretrained("finetuned_lora_smollm2")
tokenizer.save_pretrained("finetuned_lora_smollm2")

prompt = "Write a concise function in Python that reverses a string."
x = tokenizer(prompt, return_tensors="pt").to(model.device)
with torch.inference_mode():
    y = model.generate(**x, max_new_tokens=120, do_sample=True, temperature=0.7)
print(tokenizer.decode(y[0], skip_special_tokens=True))


Write a concise function in Python that reverses a string.

### Input:
Song: "Mr. Solem

### Output:
Mr. Solem

### Explanation:
Song's music is solem, and Mr. Solem is solem, and so is Mr. Solem - no, they're solem, and so am I.

### Input:
Song: "Mr. Solem"

### Output:
Mr. Solem

### Explanation:
Song's music is solem, and so is Mr. Solem - no, they're solem


RLHF with DPO

In [11]:
from datasets import load_dataset

# Anthropic HH RLHF has columns: "prompt", "chosen", "rejected"
# keep a small slice for speed; you can enlarge later
ds = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

# sanity
print(ds[0].keys())
print(ds[0]["prompt"][:200])


README.md: 0.00B [00:00, ?B/s]

harmless-base/train.jsonl.gz:   0%|          | 0.00/13.2M [00:00<?, ?B/s]

helpful-base/train.jsonl.gz:   0%|          | 0.00/16.2M [00:00<?, ?B/s]

helpful-online/train.jsonl.gz:   0%|          | 0.00/20.1M [00:00<?, ?B/s]

helpful-rejection-sampled/train.jsonl.gz:   0%|          | 0.00/25.7M [00:00<?, ?B/s]

...

helpful-rejection-sampled/test.jsonl.gz:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/160800 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/8552 [00:00<?, ? examples/s]

dict_keys(['chosen', 'rejected'])


KeyError: 'prompt'

In [12]:
from datasets import load_dataset
ds = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")
print(ds.column_names)
print(ds[0])


['chosen', 'rejected']
{'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, f

In [13]:
def add_prompt(example):
    # Simple heuristic: use first few lines of chosen as "prompt"
    chosen = example["chosen"]
    prompt_part = chosen.split("\n")[0][:200]
    example["prompt"] = prompt_part
    return example

ds = ds.map(add_prompt)
print(ds.column_names)
print(ds[0])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

['chosen', 'rejected', 'prompt']
{'chosen': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,\n\nHuman: What's your favorite one?\n\nAssistant: I haven't even thought about it.", 'rejected': "\n\nHuman: What are some cuss words in english?\n\nAssistant: Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, mas

In [14]:
from trl import DPOTrainer, DPOConfig
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 768

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)

args = DPOConfig(
    output_dir="outputs_dpo",
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    max_steps=200,
    logging_steps=10,
    save_steps=100,
    beta=0.1,
    max_prompt_length=384,
    max_length=MAX_LEN,
    learning_rate=2e-5,
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=args,
    tokenizer=tokenizer,
    train_dataset=ds,  # now has prompt/chosen/rejected
)
trainer.train()


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


Extracting prompt in train dataset (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

Tokenizing train dataset (num_proc=6):   0%|          | 0/2000 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [15]:
import os, sys, glob, site
venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands: site_dir = cands[0]
site.addsitedir(site_dir)

import torch, transformers, trl
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)
print("TRL:", trl.__version__)


Torch: 2.8.0+cu126
Transformers: 4.57.1
TRL: 0.23.0


In [16]:
from datasets import load_dataset
ds = load_dataset("Anthropic/hh-rlhf", split="train[:2000]")

def add_prompt(example):
    # quick synthetic prompt from the first line of 'chosen'
    ch = example["chosen"]
    example["prompt"] = ch.split("\n", 1)[0][:200]
    return example

ds = ds.map(add_prompt)
print(ds.column_names)  # ['chosen','rejected','prompt']


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

['chosen', 'rejected', 'prompt']


In [19]:
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 768

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,      # memory-friendly on T4
    full_finetuning = False,     # PEFT path
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [21]:
import os, sys, glob, site
venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands: site_dir = cands[0]
site.addsitedir(site_dir)

import torch, transformers, trl
print("Torch:", torch.__version__)          # expect 2.5.1+cu124 (or similar)
print("Transformers:", transformers.__version__)  # expect 4.56.2
print("TRL:", trl.__version__)              # 0.23.0 or 0.24.0 both OK


Torch: 2.8.0+cu126
Transformers: 4.57.1
TRL: 0.23.0


In [22]:
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 768

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)

model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [28]:
# 1) Make sure tokenizer has a pad token (use EOS as PAD for causal LMs)
print("Before:", tokenizer.pad_token, tokenizer.pad_token_id)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
print("After :", tokenizer.pad_token, tokenizer.pad_token_id)


Before: <|PAD_TOKEN|> 49152
After : <|PAD_TOKEN|> 49152


In [31]:
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 768

model, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)

# Ensure we have a valid PAD token (use EOS as PAD for causal LMs)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

# Add LoRA adapters
model = FastLanguageModel.get_peft_model(
    model,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [32]:
from trl import DPOTrainer, DPOConfig

args = DPOConfig(
    output_dir="outputs_dpo",
    per_device_train_batch_size=8,     # drop to 4/2 if OOM
    gradient_accumulation_steps=1,
    max_steps=200,
    logging_steps=10,
    save_steps=100,

    # These make TRL's built-in DPO collator handle padding/truncation correctly:
    max_prompt_length=384,
    max_length=MAX_LEN,

    beta=0.1,
    learning_rate=2e-5,
)

trainer = DPOTrainer(
    model=model,
    ref_model=None,
    args=args,
    tokenizer=tokenizer,
    train_dataset=ds,    # your ds has ['prompt','chosen','rejected']
)

print("✅ Trainer ready. Starting training…")
trainer.train()


The model is already on multiple devices. Skipping the move to device specified in `args`.


✅ Trainer ready. Starting training…


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,000 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 1 x 1) = 8
 "-____-"     Trainable parameters = 4,884,480 of 139,400,064 (3.50% trained)


RuntimeError: Expected tensor for argument #1 'indices' to have one of the following scalar types: Long, Int; but got torch.cuda.FloatTensor instead (while checking arguments for embedding)

In [23]:
from trl import DPOTrainer, DPOConfig

args = DPOConfig(
    output_dir="outputs_dpo",
    per_device_train_batch_size=8,     # drop to 4/2 if OOM
    gradient_accumulation_steps=1,
    max_steps=200,
    logging_steps=10,
    save_steps=100,

    # These unlock the built-in DPO collator:
    max_length=MAX_LEN,                # prompt + completion
    max_prompt_length=384,             # prompt part
    # (max_completion_length only needed for seq2seq models)

    beta=0.1,
    learning_rate=2e-5,
)


In [26]:
from dataclasses import dataclass
from typing import Any, Dict, List
import torch

base_collator = DPODataCollatorWithPadding(tokenizer)

@dataclass
class CastLongCollator:
    def __call__(self, features: List[Dict[str, Any]]) -> Dict[str, Any]:
        batch = base_collator(features)
        for k, v in list(batch.items()):
            if isinstance(v, torch.Tensor) and v.dtype not in (torch.long, torch.int64):
                batch[k] = v.long()
        return batch

collator = CastLongCollator()


GRPO (reasoning) with TRL’s GRPOTrainer

In [34]:
!/content/unsloth-venv/bin/python -m pip -q install "trl==0.24.0"
import os, sys, glob, site
venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands: site_dir = cands[0]
site.addsitedir(site_dir)

import torch, transformers, trl, unsloth
from trl import GRPOTrainer, GRPOConfig
print("TRL:", trl.__version__)


TRL: 0.23.0


In [35]:
from datasets import load_dataset

gsm = load_dataset("openai/gsm8k", "main", split="train[:1000]")  # small subset
# Fields: "question", "answer" (answer often ends with '#### final_number')
print(gsm[0])


README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

{'question': 'Natalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?', 'answer': 'Natalia sold 48/2 = <<48/2=24>>24 clips in May.\nNatalia sold 48+24 = <<48+24=72>>72 clips altogether in April and May.\n#### 72'}


In [36]:
import re

def extract_final_number(s: str):
    # common GSM8K pattern: "... #### 42"
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

def reward_function(samples, **kwargs):
    # samples: list[str] model generations
    # kwargs can include batch with "answer"
    rewards = []
    golds = kwargs.get("references")  # we’ll pass gold answers separately
    for gen, gold in zip(samples, golds):
        gold_num = extract_final_number(gold or "")
        got_num  = extract_final_number(gen or "")
        rewards.append(1.0 if (gold_num is not None and gold_num == got_num) else 0.0)
    return rewards


In [37]:
from unsloth import FastLanguageModel, FastModel

BASE_MODEL = "unsloth/smollm2-135m"
MAX_LEN = 768

policy, tokenizer = FastModel.from_pretrained(
    model_name      = BASE_MODEL,
    max_seq_length  = MAX_LEN,
    load_in_4bit    = True,
    full_finetuning = False,
)
policy = FastLanguageModel.get_peft_model(
    policy,
    r=16,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    lora_alpha=16, lora_dropout=0.05, bias="none",
    use_gradient_checkpointing="unsloth",
    max_seq_length=MAX_LEN,
)


==((====))==  Unsloth 2025.11.2: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Making `model.base_model.model.model` require gradients


In [38]:
def to_prompt(ex):
    # short, explicit instruction
    return f"""Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.

Problem:
{ex['question']}
"""

prompts = [to_prompt(ex) for ex in gsm]
references = [ex["answer"] for ex in gsm]  # used by reward function


In [43]:
# 1) Build a Dataset with prompts (and keep references separately for the reward)
from datasets import Dataset

train_ds = Dataset.from_dict({"prompt": prompts})
len(train_ds), train_ds[0]


(1000,
 {'prompt': "Solve the math word problem step by step, then answer with '#### <final_number>' on the last line.\n\nProblem:\nNatalia sold clips to 48 of her friends in April, and then she sold half as many clips in May. How many clips did Natalia sell altogether in April and May?\n"})

In [45]:
# assumes you already created `prompts` and `references` lists earlier
prompt_to_answer = {p: a for p, a in zip(prompts, references)}


In [46]:
import re

def extract_final_number(s: str):
    m = re.search(r"####\s*(-?\d+(?:\.\d+)?)", s)
    return m.group(1).strip() if m else None

def reward_function(*, prompts=None, completions=None, completion_ids=None, **kwargs):
    """
    Unsloth GRPO will call this with keyword args. We use:
      - prompts:     list[str]   (batch prompts)
      - completions: list[str]   (model generations for each prompt)
    Return: list[float] rewards length == len(completions)
    """
    rewards = []
    for p, gen in zip(prompts or [], completions or []):
        gold = prompt_to_answer.get(p, "")
        got = extract_final_number(gen or "")
        ans = extract_final_number(gold or "")
        rewards.append(1.0 if (ans is not None and got == ans) else 0.0)
    return rewards


In [2]:
from trl import GRPOTrainer, GRPOConfig
from datasets import Dataset

# If you previously switched to train_dataset style, keep it:
train_ds = Dataset.from_dict({"prompt": prompts})

config = GRPOConfig(
    output_dir="outputs_grpo",
    per_device_train_batch_size=8,    # lower if OOM
    gradient_accumulation_steps=1,
    max_steps=150,
    logging_steps=10,
    save_steps=100,
    learning_rate=5e-6,
    max_prompt_length=384,
    max_completion_length=256,
    beta=0.03,
)

# Make sure PAD token exists
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"

trainer = GRPOTrainer(
    model=policy,
    tokenizer=tokenizer,
    args=config,
    reward_funcs=[reward_function],          # <-- correct
    train_dataset=train_ds,                  # dataset with a "prompt" column
    train_kwargs={},                         # we use the prompt_to_answer lookup directly
)

print("✅ Starting GRPO training…")
trainer.train()


ModuleNotFoundError: No module named 'trl'

In [3]:
import os, sys, glob, site

venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"

# Find the venv site-packages folder automatically
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]

site.addsitedir(site_dir)

import torch, transformers
print("✅ Attached venv at:", site_dir)
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)


✅ Attached venv at: /content/unsloth-venv/lib/python3.12/site-packages
Torch: 2.8.0+cu126
Transformers: 4.57.1


In [4]:
try:
    import trl
    print("TRL version:", trl.__version__)
except ImportError:
    # Install into the venv, then reattach
    !/content/unsloth-venv/bin/python -m pip install -q "trl==0.24.0"
    import importlib, site, glob
    site.addsitedir(site_dir)
    importlib.invalidate_caches()
    import trl
    print("Installed TRL:", trl.__version__)


/bin/bash: line 1: /content/unsloth-venv/bin/python: No such file or directory


ModuleNotFoundError: No module named 'trl'

In [5]:
# Clean venv
!rm -rf /content/unsloth-venv
!python -m venv /content/unsloth-venv --without-pip
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py
!/content/unsloth-venv/bin/python get-pip.py
!/content/unsloth-venv/bin/python -m pip install --upgrade pip setuptools wheel

# Torch (CUDA 12.4 wheel), Transformers, TRL, and Unsloth + deps
!/content/unsloth-venv/bin/python -m pip install --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124
!/content/unsloth-venv/bin/python -m pip install \
  "transformers==4.56.2" "trl==0.24.0" \
  "accelerate>=0.34.2" "peft>=0.13.2" "bitsandbytes>=0.45.0" "datasets>=2.20.0" \
  unsloth unsloth_zoo


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [setuptools]
Looking in indexes: https://download.pytorch.org/whl/cu124
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.2/908.2 MB 24.1 MB/s  0:00:20
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 129.3 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.9 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 126.4 MB/s  0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 30.6 MB/s  0:00:13
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 60.5 MB/s  0:00:04
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 72.3 MB/s  0:00:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 67.3 MB/s  0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 67.9 MB/s  0:00:01
     ━━━━━━━━━━━━

In [8]:
# Remove conflicting packages from the *venv*
!/content/unsloth-venv/bin/python -m pip uninstall -y xformers torchvision

# (Optional) make sure torch & core libs are exactly what we want
!/content/unsloth-venv/bin/python -m pip install -q --index-url https://download.pytorch.org/whl/cu124 torch==2.5.1+cu124
!/content/unsloth-venv/bin/python -m pip install -q "transformers==4.56.2" "trl==0.24.0" "accelerate>=0.34.2" "peft>=0.13.2" "datasets>=2.20.0" unsloth unsloth_zoo


Found existing installation: xformers 0.0.32.post2
Uninstalling xformers-0.0.32.post2:
  Successfully uninstalled xformers-0.0.32.post2
Found existing installation: torchvision 0.23.0
Uninstalling torchvision-0.23.0:
  Successfully uninstalled torchvision-0.23.0


In [10]:
import os, sys, glob, site

venv_path = "/content/unsloth-venv"
pyver = f"{sys.version_info.major}.{sys.version_info.minor}"

# Find the venv site-packages folder automatically
site_dir = f"{venv_path}/lib/python{pyver}/site-packages"
cands = glob.glob(f"{venv_path}/lib/python*/site-packages")
if not os.path.isdir(site_dir) and cands:
    site_dir = cands[0]

site.addsitedir(site_dir)

import torch, transformers
print("✅ Attached venv at:", site_dir)
print("Torch:", torch.__version__)
print("Transformers:", transformers.__version__)

✅ Attached venv at: /content/unsloth-venv/lib/python3.12/site-packages
Torch: 2.8.0+cu126
Transformers: 4.57.1


In [1]:
# should print a GPU name and True
import torch, platform
print("CUDA available:", torch.cuda.is_available())
print("Torch:", torch.__version__, "| CUDA build:", torch.version.cuda)


CUDA available: False
Torch: 2.8.0+cu126 | CUDA build: 12.6
